In [59]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
import jinja2
from anthropic import AnthropicBedrock
from core import stages
import json


In [61]:
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [62]:
environment = jinja2.Environment()

In [63]:
tsp_tpl = environment.from_string(stages.typespec.PROMPT)
dzl_tpl = environment.from_string(stages.drizzle.PROMPT)
rtr_tpl = environment.from_string(stages.router.PROMPT)
hdl_tpl = environment.from_string(stages.handlers.PROMPT)
pre_tpl = environment.from_string(stages.processors.PROMPT_PRE)

In [64]:
application_description = """
Bot that tracks my exercise routine in the gym, tracks progress and suggests new routines
for specific list of available equipment and time constraints.
""".strip()

In [65]:
from shutil import copytree, ignore_patterns


copytree('./templates/', './app_output', ignore=ignore_patterns('*.pyc', '__pycache__')) # 'node_modules'


'./app_output'

In [66]:
prompt = tsp_tpl.render(
    application_description=application_description,
)

tsp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
typespec = stages.typespec.parse_output(tsp_response.content[0].text)

In [67]:
typespec

{'reasoning': 'For a gym tracking bot, we need to handle several aspects:\n1. Recording exercises with details like sets, reps, weight\n2. Equipment availability constraints\n3. Time tracking for workouts\n4. Progress monitoring\n5. Routine suggestions\n\nExpected message patterns:\n"I did 3 sets of bench press, 8 reps each with 175lbs"\n-> recordExercise({name: "bench press", sets: 3, reps: 8, weight: 175})\n\n"What\'s my progress on deadlifts?"\n-> getProgress({exerciseName: "deadlifts"})\n\n"Suggest a 45-minute workout with dumbbells and bench"\n-> suggestRoutine({availableEquipment: ["dumbbells", "bench"], duration: "45m"})',
 'typespec_definitions': 'model Exercise {\n  name: string;\n  sets: integer;\n  reps: integer;\n  weight: float;\n  notes: string;\n  timestamp: utcDateTime;\n}\n\nmodel Equipment {\n  name: string;\n  type: string;\n}\n\nmodel Progress {\n  exerciseName: string;\n  history: Exercise[];\n  maxWeight: float;\n  averageReps: float;\n}\n\nmodel WorkoutRoutine {\

In [68]:
print(typespec["reasoning"])

For a gym tracking bot, we need to handle several aspects:
1. Recording exercises with details like sets, reps, weight
2. Equipment availability constraints
3. Time tracking for workouts
4. Progress monitoring
5. Routine suggestions

Expected message patterns:
"I did 3 sets of bench press, 8 reps each with 175lbs"
-> recordExercise({name: "bench press", sets: 3, reps: 8, weight: 175})

"What's my progress on deadlifts?"
-> getProgress({exerciseName: "deadlifts"})

"Suggest a 45-minute workout with dumbbells and bench"
-> suggestRoutine({availableEquipment: ["dumbbells", "bench"], duration: "45m"})


In [69]:
print(typespec["typespec_definitions"])

model Exercise {
  name: string;
  sets: integer;
  reps: integer;
  weight: float;
  notes: string;
  timestamp: utcDateTime;
}

model Equipment {
  name: string;
  type: string;
}

model Progress {
  exerciseName: string;
  history: Exercise[];
  maxWeight: float;
  averageReps: float;
}

model WorkoutRoutine {
  exercises: Exercise[];
  estimatedDuration: duration;
  requiredEquipment: Equipment[];
}

interface GymBot {
  @llm_func(2)
  recordExercise(exercise: Exercise): void;

  @llm_func(3)
  getProgress(exerciseName: string): Progress;

  @llm_func(2)
  suggestRoutine(
    availableEquipment: Equipment[],
    duration: duration,
    focusArea?: string
  ): WorkoutRoutine;

  @llm_func(1)
  listWorkouts(from: utcDateTime, to: utcDateTime): Exercise[];
}


In [70]:
prompt = dzl_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
)

dzl_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
drizzle = stages.drizzle.parse_output(dzl_response.content[0].text)

In [71]:
print(drizzle["drizzle_schema"])

import { 
  pgTable, 
  text, 
  integer, 
  timestamp, 
  doublePrecision,
  primaryKey 
} from "drizzle-orm/pg-core";

export const exercisesTable = pgTable("exercises", {
  id: integer("id").primaryKey().generatedAlwaysAsIdentity(),
  name: text("name").notNull(),
  sets: integer("sets").notNull(),
  reps: integer("reps").notNull(),
  weight: doublePrecision("weight").notNull(),
  notes: text("notes"),
  timestamp: timestamp("timestamp").notNull().defaultNow()
});

export const equipmentTable = pgTable("equipment", {
  id: integer("id").primaryKey().generatedAlwaysAsIdentity(),
  name: text("name").notNull(),
  type: text("type").notNull()
});

export const workoutRoutinesTable = pgTable("workout_routines", {
  id: integer("id").primaryKey().generatedAlwaysAsIdentity(),
  estimated_duration: integer("estimated_duration").notNull(), // stored in seconds
  created_at: timestamp("created_at").notNull().defaultNow()
});

// Junction table for workout routines and exercises
export const 

In [72]:
prompt = rtr_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
    user_request="I want to record my exercise routine for today."
)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}],
    tools = stages.router.TOOLS
)

funcs = stages.router.parse_outputs([content for content in exp_response.content])["user_functions"]


In [73]:
funcs

[{'name': 'recordExercise',
  'description': 'Records a single exercise workout with details including name, sets, reps, weight, notes, and timestamp. Use this when a user wants to log their exercise activity.',
  'examples': ['I did 3 sets of bench press with 12 reps at 135 pounds',
   'Just finished 5 sets of squats, 5 reps each at 225 lbs',
   'Log my deadlift: 3x5 at 185 pounds',
   'Record my exercise: 3 sets of pull-ups, 8 reps each']},
 {'name': 'getProgress',
  'description': 'Retrieves progress history and statistics for a specific exercise, including history, maximum weight achieved, and average reps. Use when user wants to check their improvement or history for a particular exercise.',
  'examples': ['Show me my bench press progress',
   'How am I doing with my squats?',
   "What's my progress on deadlifts?",
   'Check my pull-up improvements']},
 {'name': 'suggestRoutine',
  'description': 'Suggests a workout routine based on available equipment, desired duration, and optio

In [74]:
pre_processors = {}
for function_name in typespec["llm_functions"]:
    prompt = pre_tpl.render(
        function_name=function_name,
        typespec_definitions=typespec["typespec_definitions"],
    )

    pre_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": prompt}]
    )
    pre_processor = stages.processors.parse_output(pre_response.content[0].text)
    pre_processors[function_name] = pre_processor

In [75]:
print(pre_response.content[0].text)

<instructions>
For listWorkouts function:
- Requires two UTC datetime arguments (from and to) to define the time range
- If specific times are not provided, assume time is 00:00:00
- Default date range is current day if not specified
- When a single date is mentioned, assume it's the "from" date with "to" being end of that day
- Parse natural language date references like "last week", "yesterday", "this month"
- Both dates should be in ISO 8601 format (YYYY-MM-DDTHH:mm:ssZ)
</instructions>

<examples>
    <example>
        <input>Show me my workouts from yesterday</input>
        <output>{
            "from": "2024-01-16T00:00:00Z",
            "to": "2024-01-16T23:59:59Z"
        }</output>
    </example>

    <example>
        <input>What exercises did I do between January 1st and January 15th?</input>
        <output>{
            "from": "2024-01-01T00:00:00Z",
            "to": "2024-01-15T23:59:59Z"
        }</output>
    </example>

    <example>
        <input>Show my workout h

In [76]:
pre_processors

{'recordExercise': {'instructions': 'For recordExercise function:\n- Name should be a common exercise name in lowercase\n- Sets defaults to 1 if not specified\n- If reps or weight aren\'t specified, they should be inferred from context\n- Notes can be empty string if not provided\n- Timestamp should default to current time if not specified\n- Weight should be in kilograms\n- Numbers followed by \'x\' are interpreted as reps (e.g., "80x6" means 80kg for 6 reps)',
  'examples': [('bench press 80kg for 3 sets of 8 reps',
    '{\n    "name": "bench press",\n    "sets": 3,\n    "reps": 8,\n    "weight": 80.0,\n    "notes": "",\n    "timestamp": "2024-01-20T15:30:00Z"\n}'),
   ('did squats 100x5 feeling strong today',
    '{\n    "name": "squats",\n    "sets": 1,\n    "reps": 5,\n    "weight": 100.0,\n    "notes": "feeling strong today",\n    "timestamp": "2024-01-20T15:30:00Z"\n}'),
   ('just finished 3 sets deadlift 120kg x 6',
    '{\n    "name": "deadlift",\n    "sets": 3,\n    "reps": 6

In [77]:
handlers = {}
for function_name in typespec["llm_functions"]:
    prompt = hdl_tpl.render(
        function_name=function_name,
        typespec_definitions=typespec["typespec_definitions"],
        drizzle_schema=drizzle["drizzle_schema"],
    )

    hdl_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": prompt}]
    )
    handlers[function_name] = stages.handlers.parse_output(hdl_response.content[0].text)


In [78]:
handlers

{'recordExercise': {'handler': 'import { db } from "../db";\nimport { exercisesTable, progressTable } from \'../db/schema/application\';\nimport { eq } from \'drizzle-orm\';\n\nconst handle = async (exercise: {\n    name: string;\n    sets: number;\n    reps: number;\n    weight: number;\n    notes: string;\n    timestamp: Date;\n}): Promise<void> => {\n    // Insert the new exercise record\n    await db.insert(exercisesTable).values({\n        name: exercise.name,\n        sets: exercise.sets,\n        reps: exercise.reps,\n        weight: exercise.weight,\n        notes: exercise.notes,\n        timestamp: exercise.timestamp\n    }).execute();\n\n    // Update progress table\n    const existingProgress = await db\n        .select()\n        .from(progressTable)\n        .where(eq(progressTable.exercise_name, exercise.name))\n        .execute();\n\n    if (existingProgress.length > 0) {\n        // Update existing progress\n        await db\n            .update(progressTable)\n       

In [50]:
#import os

#with open("./templates/interpolation/handler.debug.tpl", "r") as f:
#    handler_debug_tpl = environment.from_string(f.read())
#    
#    for handler_name in handlers.keys():
#        params = {
#            "handler": {"name": handler_name},
#            #"instructions": pre_processors[hdl_key]["instructions"],
#            #"examples": pre_processors[hdl_key]["examples"],#
#        }
#        file_content = handler_debug_tpl.render(**params)

#        # Convert PascalCase to snake_case for file naming
#        handler_snake_name = ''.join(['_' + c.lower() if c.isupper() else c for c in handler_name]).lstrip('_')
#        handler_file_name = f'{handler_snake_name}_handler_debug'
        
#        with open(os.path.join('./app_output/app_schema/src/handlers', handler_file_name + '.ts'), 'w') as f:
#            f.write(file_content)
            
#        handlers[handler_name]["module_name"] = handler_file_name
        
#        print(f"Handler: {handler_file_name}")
#        print(file_content)


Handler: record_exercise_handler_debug
import { GenericHandler, Message } from "../common/handler";
import { client } from "../common/llm";

const preProcessor = async (input: Message[]): Promise<[string]> => {
    return [''];
};

const handle = (input: string): string => {
    return '';
};

const postProcessor = (output: string): Message[] => {
    const content = 'handler recordExercise executed';
    return [{ role: 'assistant', content: content }];
};

export const recordExercise = new GenericHandler<[string], string>(handle, preProcessor, postProcessor);
Handler: get_progress_handler_debug
import { GenericHandler, Message } from "../common/handler";
import { client } from "../common/llm";

const preProcessor = async (input: Message[]): Promise<[string]> => {
    return [''];
};

const handle = (input: string): string => {
    return '';
};

const postProcessor = (output: string): Message[] => {
    const content = 'handler getProgress executed';
    return [{ role: 'assistant', 

In [90]:
import os

with open("./templates/interpolation/handler.tpl", "r") as f:
    handler_debug_tpl = environment.from_string(f.read())
    
    for handler_name in handlers.keys():
        params = {
            "handler_name": handler_name,
            "handler": handlers[handler_name]["handler"],
            "instructions": pre_processors[handler_name]["instructions"],
            "examples": pre_processors[handler_name]["examples"],
        }
        file_content = handler_debug_tpl.render(**params)

        # Convert PascalCase to snake_case for file naming
        handler_snake_name = ''.join(['_' + c.lower() if c.isupper() else c for c in handler_name]).lstrip('_')
        
        with open(os.path.join('./app_output/app_schema/src/handlers', handler_snake_name + '.ts'), 'w') as f:
            f.write(file_content)
            
        handlers[handler_name]["module_name"] = handler_snake_name
        
        print(f"Handler: {handler_snake_name}")
        #print(file_content)

Handler: record_exercise
Handler: get_progress
Handler: suggest_routine
Handler: list_workouts


In [82]:
handlers

{'recordExercise': {'handler': 'import { db } from "../db";\nimport { exercisesTable, progressTable } from \'../db/schema/application\';\nimport { eq } from \'drizzle-orm\';\n\nconst handle = async (exercise: {\n    name: string;\n    sets: number;\n    reps: number;\n    weight: number;\n    notes: string;\n    timestamp: Date;\n}): Promise<void> => {\n    // Insert the new exercise record\n    await db.insert(exercisesTable).values({\n        name: exercise.name,\n        sets: exercise.sets,\n        reps: exercise.reps,\n        weight: exercise.weight,\n        notes: exercise.notes,\n        timestamp: exercise.timestamp\n    }).execute();\n\n    // Update progress table\n    const existingProgress = await db\n        .select()\n        .from(progressTable)\n        .where(eq(progressTable.exercise_name, exercise.name))\n        .execute();\n\n    if (existingProgress.length > 0) {\n        // Update existing progress\n        await db\n            .update(progressTable)\n       

In [83]:
with open("./templates/interpolation/logic_index.tpl", "r") as f:
    logic_index_tpl = environment.from_string(f.read())
    
    params = {
        "handlers": handlers
    }
    file_content = logic_index_tpl.render(**params)
    
    with open(os.path.join('./app_output/app_schema/src/logic/index.ts'), 'w') as f:
        f.write(file_content)
            
    print(file_content)

import { GenericHandler } from "../common/handler";

import { recordExercise } from "../handlers/record_exercise";

import { getProgress } from "../handlers/get_progress";

import { suggestRoutine } from "../handlers/suggest_routine";

import { listWorkouts } from "../handlers/list_workouts";


export const handlers: {[key: string]: GenericHandler<any[], any>} = {
    
    'recordExercise': recordExercise,
    
    'getProgress': getProgress,
    
    'suggestRoutine': suggestRoutine,
    
    'listWorkouts': listWorkouts,
    
};


In [84]:
with open("./templates/interpolation/router.tpl", "r") as f:
    logic_index_tpl = environment.from_string(f.read())
    
    params = {
        "handlers": handlers
    }
    file_content = logic_index_tpl.render(**params)
    
    with open(os.path.join('./app_output/app_schema/src/logic/router.ts'), 'w') as f:
        f.write(file_content)
            
    print(file_content)

import { client } from "../common/llm";
import { Message } from "../common/handler";
const nunjucks = require("nunjucks");

const router_prompt: string = `
Based on converstation between user and assistant determine which function should
handle current message based on function description and message content.
{% for function in functions%}
<function name="{{function.name}}">
    <description>{{function.description}}</description>
    {% for example in function.examples %}
    <example>{{example}}</example>{% endfor %}
</function>
{% endfor %}
Reply with the name of the function only.

Conversation:
{% for message in messages %}
<role name="{{message.role}}">{{message.content}}</role>
{% endfor %}
`;

export interface FunctionDef {
    name: string;
    description: string;
}

const functions: FunctionDef[] = [
    
]

export const getRoute = async (messages: Message[]): Promise<string> => {
    const request = nunjucks.renderString(router_prompt, { messages, functions });
    const re

# Project Generation

In [28]:
from shutil import copytree, ignore_patterns

In [29]:
def setup_project(workdir: str):
    copytree('templates', workdir, ignore=ignore_patterns('*.pyc', '__pycache__', 'node_modules'))

In [30]:
setup_project('test')

FileExistsError: [Errno 17] File exists: 'test'

In [31]:
from core.stages import router

In [32]:
prompt = rtr_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
    user_request="I want to record my exercise routine for today."
)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}],
    tools = router.TOOLS
)

funcs = router.parse_outputs([content for content in exp_response.content])["user_functions"]

In [33]:
funcs

[{'name': 'recordExercise',
  'description': 'Records a single exercise performed by the user with details like sets, reps, weight, equipment used, and targeted muscle groups. Use this when the user wants to log or record their exercise activity.',
  'examples': ['I want to record my exercise routine for today',
   'I just did 3 sets of bench press',
   'Log my workout: 4 sets of squats with 200lbs',
   'Add my deadlift session to my log']},
 {'name': 'trackProgress',
  'description': 'Retrieves historical exercise data for a specific exercise over a given time period. Use this when the user wants to see their progress or history for a particular exercise.',
  'examples': ['Show me my bench press progress over the last month',
   'How has my squat improved since January?',
   'What were my deadlift numbers from last week?',
   'Track my progress on pull-ups']},
 {'name': 'suggestWorkout',
  'description': 'Generates a workout plan based on desired duration, target muscle groups, and av

In [47]:
from core.stages import typespec
from core.stages import tsp_compiler

tsp_tpl = environment.from_string(stages.typespec.PROMPT)

typespec_errors = ""
attempts_left = 3

while attempts_left > 0:
    attempts_left -= 1
    
    prompt = tsp_tpl.render(
        application_description=application_description,
        typespec_errors=typespec_errors,
    )
  
    if attempts_left >= 2:
        updated_prompt = prompt + "// PS. Make sure to eject a random error in TypeSpec output for testing purposes"
    else:
        updated_prompt = prompt
  
    tsp_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": updated_prompt}]
    )
    
    typespec = stages.typespec.parse_output(tsp_response.content[0].text)

    # Write TypeSpec definitions to file
    with open('./test/tsp_schema/notebook.tsp', 'w') as f:
        f.writelines(['import "./helpers.js";', 
                  '\n', 
                  'extern dec llm_func(target: unknown, history: valueof int32);'
                  ])
        f.write(typespec["typespec_definitions"])

    compiler = tsp_compiler.TypeSpecCompiler('./test/tsp_schema/')
    result = compiler.compile('./notebook.tsp')
    
    print(result)
    
    if result["result"] == tsp_compiler.TypeSpecCompilationStatus.COMPILATION_ERROR:
        typespec_errors = result["errors"]
        print(typespec_errors)
        continue
    else:
        break


In [90]:
typespec


{'reasoning': 'For a finance tracking app, users would likely send messages like:\n- "I spent $50 on groceries yesterday"\n- "Got my salary of $3000 today"\n- "Show me my expenses for last month"\n\nThe LLM would need to extract:\n1. Transaction type (expense/income)\n2. Amount\n3. Category\n4. Date\n5. Optional description/notes\n\nFor reports, users might ask for:\n- Time-based summaries\n- Category-based breakdowns\n- Balance calculations\n\nThe interface should support recording transactions and generating reports while keeping the argument structure simple enough to be extracted from natural language.',
 'typespec_definitions': 'model Transaction {\n  type: string; // "expense" or "income"\n  amount: decimal;\n  category: string;\n  date: utcDateTime;\n  description: string;\n}\n\nmodel TransactionSummary {\n  totalIncome: decimal;\n  totalExpenses: decimal;\n  balance: decimal;\n  transactions: Transaction[];\n}\n\ninterface FinanceTracker {\n  @llm_func(1)\n  recordTransaction(t

In [ ]:
prompt = exp_tpl.render(application_description=application_description)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
expansion = stages.expansion.parse_output(exp_response.content[0].text)

In [ ]:
print(expansion["application_specification"])

<types>
        <type>exercise
            - name
            - equipment_required
            - sets
            - reps
            - weight</type>
        <type>equipment
            - name
            - type</type>
        <type>workout_session
            - date
            - duration
            - exercises_performed</type>
        <type>routine
            - exercises
            - duration
            - equipment_needed</type>
    </types>
    <operations>
        <operation>record_workout_session
            - date
            - exercises with sets/reps/weights</operation>
        <operation>track_progress
            - exercise
            - time period</operation>
        <operation>list_available_equipment</operation>
        <operation>generate_routine
            - available equipment
            - time constraint
            - fitness level</operation>
        <operation>view_history
            - time period</operation>
    </operations>
